In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Driver Debug Printing

When working with a model, it may sometimes be helpful to print out the design variables, constraints, and objectives as the `Driver` iterates. OpenMDAO provides options on the Driver to let you do that.


## Driver Options

In [ ]:
import openmdao.api as om
om.show_options_table("openmdao.core.driver.Driver")

## Usage

This example shows how to use the `Driver` debug printing options. The `debug_print` option is a list of strings. Valid strings include ‘desvars’, ‘ln_cons’, ‘nl_cons’, and ‘objs’. Note that the values for the design variables printed are unscaled, physical values.

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src66", get_code("openmdao.test_suite.components.paraboloid.Paraboloid"), display=False)

:::{Admonition} `Paraboloid` class definition 
:class: dropdown

{glue:}`code_src66`
:::

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.paraboloid import Paraboloid

prob = om.Problem()
model = prob.model

model.add_subsystem('comp', Paraboloid(), promotes=['*'])
model.add_subsystem('con', om.ExecComp('c = - x + y'), promotes=['*'])

model.set_input_defaults('x', 50.0)
model.set_input_defaults('y', 50.0)

prob.set_solver_print(level=0)

prob.driver = om.ScipyOptimizeDriver()
prob.driver.options['optimizer'] = 'SLSQP'
prob.driver.options['tol'] = 1e-9
prob.driver.options['disp'] = False

prob.driver.options['debug_print'] = ['desvars','ln_cons','nl_cons','objs']

model.add_design_var('x', lower=-50.0, upper=50.0)
model.add_design_var('y', lower=-50.0, upper=50.0)
model.add_objective('f_xy')
model.add_constraint('c', upper=-15.0)

prob.setup()

prob.run_driver()


We can also use the debug printing to print some basic information about the derivative calculations so that you can see which derivative is being solved, how long it takes, and the computed values by including the ‘totals’ string in the “debug_print” list.

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.paraboloid import Paraboloid

prob = om.Problem()
model = prob.model

model.add_subsystem('comp', Paraboloid(), promotes=['*'])
model.add_subsystem('con', om.ExecComp('c = - x + y'), promotes=['*'])

model.set_input_defaults('x', 50.0)
model.set_input_defaults('y', 50.0)

prob.set_solver_print(level=0)

prob.driver = om.ScipyOptimizeDriver()
prob.driver.options['optimizer'] = 'SLSQP'
prob.driver.options['tol'] = 1e-9
prob.driver.options['disp'] = False

prob.driver.options['debug_print'] = ['totals']

model.add_design_var('x', lower=-50.0, upper=50.0)
model.add_design_var('y', lower=-50.0, upper=50.0)
model.add_objective('f_xy')
model.add_constraint('c', upper=-15.0)

prob.setup()

prob.run_driver()